# Task 2 – Data Modeling & Transformation


## 📖 Overview

In this task, we’ll transform our raw JSON dumps into a clean, dimensional data warehouse in PostgreSQL using dbt. We’ll build layered models—from raw loading scripts through staging tables to final star-schema marts—complete with testing and documentation

---

## 🎯 Objectives

1. **Load raw JSON**  
   - Write a Python script to bulk-load `data/raw/telegram_messages` JSON into a `raw.telegram_messages` schema in PostgreSQL.  
2. **Initialize dbt**  
   - Install dbt and the Postgres adapter.  
   - Run `dbt init your_project_name` and configure `profiles.yml` to point at your warehouse.  
3. **Develop Staging Models**  
   - Create `models/staging/stg_telegram_messages.sql` to:  
     - Cast JSON fields to proper types (e.g., timestamps).  
     - Rename columns for clarity (e.g., `message_id`, `text`, `has_media`).  
     - Extract nested JSON metadata (e.g., `media.file_id`, `media.file_size`).  
4. **Build Data Marts (Star Schema)**  
   - **Dimensions**  
     - `models/marts/dim_channels.sql`: one row per channel with metadata (name, URL).  
     - `models/marts/dim_dates.sql`: calendar table for time-based analysis.  
   - **Fact**  
     - `models/marts/fct_messages.sql`: one row per message, foreign keys to `dim_channels` and `dim_dates`, plus measures like `message_length`, `has_image`:contentReference[oaicite:1]{index=1}.  
5. **Testing & Documentation**  
   - Add dbt `tests` for:  
     - Uniqueness of primary keys (`message_id`, `channel_id`).  
     - Non-null critical fields (`text`, `timestamp`).  
     - At least one custom test (e.g., no messages with empty text but `has_media = FALSE`).  
   - Run `dbt docs generate` and review lineage & schema in the docs site.  
6. **Run & Validate**  
   - Execute `dbt run --models staging+ marts+`.  
   - Use `dbt test` to ensure all tests pass.  
   - Inspect the generated docs (`dbt docs serve`) to confirm model structure.

---





In [5]:
import sys
import os

# Go two levels up from the notebook to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))

# Join the path to 'src'
src_path = os.path.join(project_root, "src")

# Add 'src' to Python path
if src_path not in sys.path:
    sys.path.append(src_path)

# Confirm it's added
print("src path added:", src_path)


src path added: c:\Users\ABC\Desktop\10Acadamy\week_7\Shipping-a-Data-Product\src


In [6]:
# 2️⃣ Load .env and set PYTHONPATH
import os, sys
from dotenv import load_dotenv
from pathlib import Path

project_root = Path().cwd().parent / "Shipping-a-Data-Product"
load_dotenv(dotenv_path=project_root / ".env")
sys.path.append(str(project_root / "src"))


In [7]:
# 3️⃣ Run raw_loader to ingest JSON & images
from raw_loader import load_messages, load_images

# Load yesterday’s (or today’s) partitions
load_messages(date_str="2025-07-13")
load_images(date_str="2025-07-13")


In [8]:
# 4️⃣ Install & run dbt to build staging + marts
%env DBT_PROFILES_DIR=./dbt_project
!dbt deps   # fetches any packages
!dbt seed   # if you have any seeds
!dbt run    # builds models in analytics schema
!dbt test   # runs built-in tests


env: DBT_PROFILES_DIR=./dbt_project


Usage: dbt deps [OPTIONS]
Try 'dbt deps -h' for help.

Error: Got unexpected extra arguments (# fetches any packages)
Usage: dbt seed [OPTIONS]
Try 'dbt seed -h' for help.

Error: Invalid value for '--profiles-dir': Path './dbt_project' does not exist.
Usage: dbt run [OPTIONS]
Try 'dbt run -h' for help.

Error: Invalid value for '--profiles-dir': Path './dbt_project' does not exist.
Usage: dbt test [OPTIONS]
Try 'dbt test -h' for help.

Error: Invalid value for '--profiles-dir': Path './dbt_project' does not exist.
